## Intro

Den här notebooken tar en xml-fil från Livsmedelsverket som innehåller en klassificering av alla livsmedel. 

Exemplet utgår ifrån https://docs.python.org/3/library/xml.etree.elementtree.html 

Syftet är att göra en array med livsmedelsnummer och "huvudgrupp" som sen kan läsas ner i databasen.

I slutet läses klassificeringen ner i databasen, och namnen på klasserna hamnar i en csv-fil, med antalet livsmedel i varje klass.

Ett livsmedel har den här strukturen i xml-filen:

```<Livsmedel>
    <Nummer>1082</Nummer>
    <Namn>Skinksås</Namn>
    <ViktGram>100</ViktGram>
    <Huvudgrupp>Sås dressing majonnäs </Huvudgrupp>
    <Klassificeringar>
        <Klassificering>
            <Namn>A Gruppindelning EuroFIR</Namn>
            <Varden>
                <KlassificeringVarde>
                    <Namn>Sås</Namn>
                    <Kod>A0862</Kod>
                </KlassificeringVarde>
            </Varden>
        </Klassificering>
        <Klassificering>
            <Namn>B Artklassificering</Namn>
            <Varden>
                <KlassificeringVarde>
                    <Namn>Ko</Namn>
                    <Kod>B1201</Kod>
                </KlassificeringVarde>
            </Varden>
        </Klassificering>
        <Klassificering>
            <Namn>C Del av växt/djur</Namn>
            <Varden>
                <KlassificeringVarde>
                    <Namn>Mjölk</Namn>
                    <Kod>C0235</Kod>
                </KlassificeringVarde>
            </Varden>
        </Klassificering>
        <Klassificering>
            <Namn>E Fysisk form</Namn>
            <Varden>
                <KlassificeringVarde>
                    <Namn>Vätska, hög viskositet, med fasta bitar</Namn>
                    <Kod>E0138</Kod>
                </KlassificeringVarde>
            </Varden>
        </Klassificering>
        <Klassificering>
            <Namn>F Värmebehandling</Namn>
            <Varden>
                <KlassificeringVarde>
                    <Namn>Fullständigt upphettad </Namn>
                    <Kod>F0014</Kod>
                </KlassificeringVarde>
            </Varden>
        </Klassificering>
        <Klassificering>
            <Namn>G Tillagning</Namn>
            <Varden>
                <KlassificeringVarde>
                    <Namn>Kokt</Namn>
                    <Kod>G0014</Kod>
                </KlassificeringVarde>
            </Varden>
        </Klassificering>
        <Klassificering>
            <Namn>H Industriell process</Namn>
            <Varden>
                <KlassificeringVarde>
                    <Namn>Ingrediens tillförd</Namn>
                    <Kod>H0225</Kod>
                </KlassificeringVarde>
            </Varden>
        </Klassificering>
        <Klassificering>
            <Namn>J Konserveringsmetod</Namn>
            <Varden>
                <KlassificeringVarde>
                    <Namn>Ej konserverad</Namn>
                    <Kod>J0003</Kod>
                </KlassificeringVarde>
            </Varden>
        </Klassificering>
        <Klassificering>
            <Namn>K Förpackningsmedium</Namn>
            <Varden>
                <KlassificeringVarde>
                    <Namn>Utan förpackningsmedium</Namn>
                    <Kod>K0003</Kod>
                </KlassificeringVarde>
            </Varden>
        </Klassificering>
        <Klassificering>
            <Namn>M Förpackningstyp</Namn>
            <Varden>
                <KlassificeringVarde>
                    <Namn>Opaketerad produkt</Namn>
                    <Kod>M0003</Kod>
                </KlassificeringVarde>
            </Varden>
        </Klassificering>
        <Klassificering>
            <Namn>N Förpackningsmaterial</Namn>
            <Varden>
                <KlassificeringVarde>
                    <Namn>Oförpackad</Namn>
                    <Kod>N0003</Kod>
                </KlassificeringVarde>
            </Varden>
        </Klassificering>
        <Klassificering>
            <Namn>P Etikettinfo</Namn>
            <Varden>
                <KlassificeringVarde>
                    <Namn>Människoföda</Namn>
                    <Kod>P0026</Kod>
                </KlassificeringVarde>
            </Varden>
        </Klassificering>
        <Klassificering>
            <Namn>R Land/område</Namn>
            <Varden>
                <KlassificeringVarde>
                    <Namn>Sverige</Namn>
                    <Kod>R0376</Kod>
                </KlassificeringVarde>
            </Varden>
        </Klassificering>
    </Klassificeringar>
</Livsmedel>```

Filen börjar med lite extra innan första livsmedlet kommer, så därför blir xml-trädets rot ett steg "ner".

```<?xml version="1.0" encoding="UTF-8"?>
    <LivsmedelDataset xmlns:xsd="http://www.w3.org/2001/XMLSchema" xmlns:xsi="http://www.w3.org/2001/XMLSchema-instance">
	<Version>2017-03-14T15:37:15.533</Version>
	<LivsmedelsLista>
		<Livsmedel>```

In [1]:
import numpy as np
import sqlite3 
import xml.etree.ElementTree as ET
tree = ET.parse('metadata_20171106.xml')
root = tree.getroot()[1]
# Hade det stått getroot()[0] hade man i stället fångat trädet som är Version...

In [2]:
root.tag

'LivsmedelsLista'

In [3]:
#För att testa att vi fått tag på rätt nivå kan vi skriva ut första "barnet" i root.
for child in root[0]:
    print(child.tag,child.text)
#Anledningen till att Klassificeringar blir None är att det är en stor struktur som 
#inte låter sig skrivas som "text"

Nummer 1
Namn Talg nöt
ViktGram 100
Huvudgrupp Övrigt fett (ister, talg, kokosfett)
Klassificeringar None


In [4]:
root[0][0].text #Man borde kunna gå igenom root snyggare än nedan...

'1'

In [9]:
b = np.array(root) 

i = 0
for Livsmedel in root.findall('Livsmedel'):
    number = Livsmedel.find('Nummer').text
    group = Livsmedel.find('Huvudgrupp').text
    b[i][0] = number
    b[i][1] = group.rstrip().replace('"','*')
    i += 1

b = b[:,:2] #Kapa allt utom de två första kolumnerna
print(b)

[['1' 'Övrigt fett (ister, talg, kokosfett)']
 ['2' 'Övrigt fett (ister, talg, kokosfett)']
 ['3' 'Övrigt fett (ister, talg, kokosfett)']
 ..., 
 ['5973' 'Kryddor']
 ['5974' 'Mjukt bröd']
 ['6053' 'Nötter frön']]


## Läs ner i databasen

In [10]:
conn = sqlite3.connect('../livs.db')  # Create db and establish connection
conn.row_factory = sqlite3.Row
curs = conn.cursor()

columnObject = []

#Kolla om Huvudgrupp-kolumnen redan finns, annars lägg till den        
for row in conn.execute('PRAGMA table_info (livs);'):
    columnObject.append(row)
columns = np.array(columnObject)
newColumn = 'Huvudgrupp'
addColumn = True
for row in columns:
    print (row)
    if newColumn == row[1]:
        addColumn = False
        print (addColumn, newColumn)
if addColumn:
    #Man får se upp extra noga med kolumntypen, för den går inte att ändra i efterhand i sqlite...
    conn.execute('ALTER TABLE livs ADD COLUMN Huvudgrupp TEXT;')
    conn.commit()

conn.close()

[0 'Livsmedelsnamn' 'TEXT' 0 None 0]
[1 'Livsmedelsnummer' 'TEXT' 0 None 0]
[2 'Energi_kcal' 'REAL' 0 None 0]
[3 'Energi_kJ' 'REAL' 0 None 0]
[4 'Kolhydrater_g' 'REAL' 0 None 0]
[5 'Fett_g' 'REAL' 0 None 0]
[6 'Protein_g' 'REAL' 0 None 0]
[7 'Fibrer_g' 'REAL' 0 None 0]
[8 'Vatten_g' 'REAL' 0 None 0]
[9 'Alkohol_g' 'REAL' 0 None 0]
[10 'Aska_g' 'REAL' 0 None 0]
[11 'Monosackarider_g' 'REAL' 0 None 0]
[12 'Disackarider_g' 'REAL' 0 None 0]
[13 'Sackaros_g' 'REAL' 0 None 0]
[14 'Fullkorn_totalt_g' 'REAL' 0 None 0]
[15 'Sockerarter_g' 'REAL' 0 None 0]
[16 'Summa_mattade_fettsyror_g' 'REAL' 0 None 0]
[17 'Fettsyra_4_0_10_0_g' 'REAL' 0 None 0]
[18 'Fettsyra_12_0_g' 'REAL' 0 None 0]
[19 'Fettsyra_14_0_g' 'REAL' 0 None 0]
[20 'Fettsyra_16_0_g' 'REAL' 0 None 0]
[21 'Fettsyra_18_0_g' 'REAL' 0 None 0]
[22 'Fettsyra_20_0_g' 'REAL' 0 None 0]
[23 'Summa_enkelomattade_fettsyror_g' 'REAL' 0 None 0]
[24 'Fettsyra_16_1_g' 'REAL' 0 None 0]
[25 'Fettsyra_18_1_g' 'REAL' 0 None 0]
[26 'Summa_fleromattade_fet

In [11]:
conn = sqlite3.connect('../livs.db')  # Create db and establish connection
conn.row_factory = sqlite3.Row
curs = conn.cursor()

print (len(b))

for row in b:
    #print(row[1], row[0])
    
    #.rstrip() på nästa rad tar bort blanksteg i slutet på huvudgruppernas namn 
    #.replace('"','*') ersätter citationstecken med stjärn-tecken 
    sql = 'UPDATE livs SET Huvudgrupp = "'+row[1]+'" WHERE Livsmedelsnummer = "'+row[0]+'";'
    print(sql)
    curs.execute(sql)
    conn.commit()

conn.close()

2085
UPDATE livs SET Huvudgrupp = "Övrigt fett (ister, talg, kokosfett)" WHERE Livsmedelsnummer = "1";
UPDATE livs SET Huvudgrupp = "Övrigt fett (ister, talg, kokosfett)" WHERE Livsmedelsnummer = "2";
UPDATE livs SET Huvudgrupp = "Övrigt fett (ister, talg, kokosfett)" WHERE Livsmedelsnummer = "3";
UPDATE livs SET Huvudgrupp = "Övrigt fett (ister, talg, kokosfett)" WHERE Livsmedelsnummer = "4";
UPDATE livs SET Huvudgrupp = "Hård matfettsblandning" WHERE Livsmedelsnummer = "5";
UPDATE livs SET Huvudgrupp = "Hård matfettsblandning" WHERE Livsmedelsnummer = "6";
UPDATE livs SET Huvudgrupp = "Hård matfettsblandning" WHERE Livsmedelsnummer = "7";
UPDATE livs SET Huvudgrupp = "Hård matfettsblandning" WHERE Livsmedelsnummer = "8";
UPDATE livs SET Huvudgrupp = "Hård matfettsblandning" WHERE Livsmedelsnummer = "9";
UPDATE livs SET Huvudgrupp = "Flytande matfettsblandning" WHERE Livsmedelsnummer = "10";
UPDATE livs SET Huvudgrupp = "Hård matfettsblandning" WHERE Livsmedelsnummer = "12";
UPDATE li

UPDATE livs SET Huvudgrupp = "Hårt bröd" WHERE Livsmedelsnummer = "170";
UPDATE livs SET Huvudgrupp = "Hårt bröd" WHERE Livsmedelsnummer = "171";
UPDATE livs SET Huvudgrupp = "Hårt bröd" WHERE Livsmedelsnummer = "172";
UPDATE livs SET Huvudgrupp = "Hårt bröd" WHERE Livsmedelsnummer = "173";
UPDATE livs SET Huvudgrupp = "Hårt bröd" WHERE Livsmedelsnummer = "174";
UPDATE livs SET Huvudgrupp = "Hårt bröd" WHERE Livsmedelsnummer = "175";
UPDATE livs SET Huvudgrupp = "Hårt bröd" WHERE Livsmedelsnummer = "178";
UPDATE livs SET Huvudgrupp = "Hårt bröd" WHERE Livsmedelsnummer = "180";
UPDATE livs SET Huvudgrupp = "Hårt bröd" WHERE Livsmedelsnummer = "181";
UPDATE livs SET Huvudgrupp = "Hårt bröd" WHERE Livsmedelsnummer = "182";
UPDATE livs SET Huvudgrupp = "Hårt bröd" WHERE Livsmedelsnummer = "183";
UPDATE livs SET Huvudgrupp = "Mjukt bröd" WHERE Livsmedelsnummer = "184";
UPDATE livs SET Huvudgrupp = "Mjukt bröd" WHERE Livsmedelsnummer = "185";
UPDATE livs SET Huvudgrupp = "Mjukt bröd" WHERE L

UPDATE livs SET Huvudgrupp = "Baljväxter (bönor, linser och ärter)" WHERE Livsmedelsnummer = "331";
UPDATE livs SET Huvudgrupp = "Baljväxter (bönor, linser och ärter)" WHERE Livsmedelsnummer = "332";
UPDATE livs SET Huvudgrupp = "Svamp" WHERE Livsmedelsnummer = "333";
UPDATE livs SET Huvudgrupp = "Grönsaker" WHERE Livsmedelsnummer = "334";
UPDATE livs SET Huvudgrupp = "Grönsaker" WHERE Livsmedelsnummer = "335";
UPDATE livs SET Huvudgrupp = "Grönsaker" WHERE Livsmedelsnummer = "336";
UPDATE livs SET Huvudgrupp = "Grönsaker" WHERE Livsmedelsnummer = "337";
UPDATE livs SET Huvudgrupp = "Grönsaker" WHERE Livsmedelsnummer = "338";
UPDATE livs SET Huvudgrupp = "Grönsaker" WHERE Livsmedelsnummer = "339";
UPDATE livs SET Huvudgrupp = "Grönsaker" WHERE Livsmedelsnummer = "340";
UPDATE livs SET Huvudgrupp = "Grönsaker" WHERE Livsmedelsnummer = "342";
UPDATE livs SET Huvudgrupp = "Grönsaker" WHERE Livsmedelsnummer = "343";
UPDATE livs SET Huvudgrupp = "Grönsaker" WHERE Livsmedelsnummer = "344";
U

UPDATE livs SET Huvudgrupp = "Soppa mat" WHERE Livsmedelsnummer = "484";
UPDATE livs SET Huvudgrupp = "Soppa mat" WHERE Livsmedelsnummer = "486";
UPDATE livs SET Huvudgrupp = "Grönsaker" WHERE Livsmedelsnummer = "487";
UPDATE livs SET Huvudgrupp = "Grönsaks- rotfrukts- baljväxträtter och produkter" WHERE Livsmedelsnummer = "488";
UPDATE livs SET Huvudgrupp = "Grönsaker" WHERE Livsmedelsnummer = "489";
UPDATE livs SET Huvudgrupp = "Grönsaker" WHERE Livsmedelsnummer = "490";
UPDATE livs SET Huvudgrupp = "Grönsaker" WHERE Livsmedelsnummer = "491";
UPDATE livs SET Huvudgrupp = "Grönsaker" WHERE Livsmedelsnummer = "492";
UPDATE livs SET Huvudgrupp = "Soppa mat" WHERE Livsmedelsnummer = "493";
UPDATE livs SET Huvudgrupp = "Soppa mat" WHERE Livsmedelsnummer = "494";
UPDATE livs SET Huvudgrupp = "Grönsaker" WHERE Livsmedelsnummer = "495";
UPDATE livs SET Huvudgrupp = "Soppa mat" WHERE Livsmedelsnummer = "496";
UPDATE livs SET Huvudgrupp = "Soppa mat" WHERE Livsmedelsnummer = "497";
UPDATE livs

UPDATE livs SET Huvudgrupp = "Fruktjuice mm" WHERE Livsmedelsnummer = "641";
UPDATE livs SET Huvudgrupp = "Fruktjuice mm" WHERE Livsmedelsnummer = "642";
UPDATE livs SET Huvudgrupp = "Fruktjuice mm" WHERE Livsmedelsnummer = "643";
UPDATE livs SET Huvudgrupp = "Fruktjuice mm" WHERE Livsmedelsnummer = "645";
UPDATE livs SET Huvudgrupp = "Fruktjuice mm" WHERE Livsmedelsnummer = "646";
UPDATE livs SET Huvudgrupp = "Fruktjuice mm" WHERE Livsmedelsnummer = "647";
UPDATE livs SET Huvudgrupp = "Fruktjuice mm" WHERE Livsmedelsnummer = "649";
UPDATE livs SET Huvudgrupp = "Fruktjuice mm" WHERE Livsmedelsnummer = "652";
UPDATE livs SET Huvudgrupp = "Gröt" WHERE Livsmedelsnummer = "654";
UPDATE livs SET Huvudgrupp = "Gröt" WHERE Livsmedelsnummer = "655";
UPDATE livs SET Huvudgrupp = "Fruktjuice mm" WHERE Livsmedelsnummer = "657";
UPDATE livs SET Huvudgrupp = "Fruktjuice mm" WHERE Livsmedelsnummer = "658";
UPDATE livs SET Huvudgrupp = "Fruktjuice mm" WHERE Livsmedelsnummer = "659";
UPDATE livs SET H

...

UPDATE livs SET Huvudgrupp = "Sylt marmelad gelé äppelmos o dyl" WHERE Livsmedelsnummer = "1795";
UPDATE livs SET Huvudgrupp = "Sylt marmelad gelé äppelmos o dyl" WHERE Livsmedelsnummer = "1796";
UPDATE livs SET Huvudgrupp = "Sylt marmelad gelé äppelmos o dyl" WHERE Livsmedelsnummer = "1797";
UPDATE livs SET Huvudgrupp = "Sylt marmelad gelé äppelmos o dyl" WHERE Livsmedelsnummer = "1798";
UPDATE livs SET Huvudgrupp = "Sylt marmelad gelé äppelmos o dyl" WHERE Livsmedelsnummer = "1800";
UPDATE livs SET Huvudgrupp = "Sylt marmelad gelé äppelmos o dyl" WHERE Livsmedelsnummer = "1801";
UPDATE livs SET Huvudgrupp = "Sylt marmelad gelé äppelmos o dyl" WHERE Livsmedelsnummer = "1802";
UPDATE livs SET Huvudgrupp = "Sylt marmelad gelé äppelmos o dyl" WHERE Livsmedelsnummer = "1803";
UPDATE livs SET Huvudgrupp = "Sylt marmelad gelé äppelmos o dyl" WHERE Livsmedelsnummer = "1804";
UPDATE livs SET Huvudgrupp = "Sylt marmelad gelé äppelmos o dyl" WHERE Livsmedelsnummer = "1805";
UPDATE livs SET Huvu

UPDATE livs SET Huvudgrupp = "Kaffe" WHERE Livsmedelsnummer = "1957";
UPDATE livs SET Huvudgrupp = "Kaffe" WHERE Livsmedelsnummer = "1958";
UPDATE livs SET Huvudgrupp = "Kaffe" WHERE Livsmedelsnummer = "1959";
UPDATE livs SET Huvudgrupp = "Te" WHERE Livsmedelsnummer = "1960";
UPDATE livs SET Huvudgrupp = "Te" WHERE Livsmedelsnummer = "1962";
UPDATE livs SET Huvudgrupp = "Te" WHERE Livsmedelsnummer = "1963";
UPDATE livs SET Huvudgrupp = "Te" WHERE Livsmedelsnummer = "1964";
UPDATE livs SET Huvudgrupp = "Ättika vinäger" WHERE Livsmedelsnummer = "1965";
UPDATE livs SET Huvudgrupp = "Ättika vinäger" WHERE Livsmedelsnummer = "1966";
UPDATE livs SET Huvudgrupp = "Ättika vinäger" WHERE Livsmedelsnummer = "1967";
UPDATE livs SET Huvudgrupp = "Senap ketchup HP-sås soja *smaksättare*" WHERE Livsmedelsnummer = "1968";
UPDATE livs SET Huvudgrupp = "Senap ketchup HP-sås soja *smaksättare*" WHERE Livsmedelsnummer = "1969";
UPDATE livs SET Huvudgrupp = "Senap ketchup HP-sås soja *smaksättare*" WHERE 

UPDATE livs SET Huvudgrupp = "Pizza paj pirog färdig smörgås" WHERE Livsmedelsnummer = "2138";
UPDATE livs SET Huvudgrupp = "Sås dressing majonnäs" WHERE Livsmedelsnummer = "2142";
UPDATE livs SET Huvudgrupp = "Sås dressing majonnäs" WHERE Livsmedelsnummer = "2144";
UPDATE livs SET Huvudgrupp = "Efterrätter" WHERE Livsmedelsnummer = "2147";
UPDATE livs SET Huvudgrupp = "Grönsaks- rotfrukts- baljväxträtter och produkter" WHERE Livsmedelsnummer = "2160";
UPDATE livs SET Huvudgrupp = "Svamp" WHERE Livsmedelsnummer = "2161";
UPDATE livs SET Huvudgrupp = "Rotfrukter" WHERE Livsmedelsnummer = "2162";
UPDATE livs SET Huvudgrupp = "Sås dressing majonnäs" WHERE Livsmedelsnummer = "2163";
UPDATE livs SET Huvudgrupp = "Grönsaker" WHERE Livsmedelsnummer = "2164";
UPDATE livs SET Huvudgrupp = "Potatisprodukter o rätter" WHERE Livsmedelsnummer = "2165";
UPDATE livs SET Huvudgrupp = "Sås dressing majonnäs" WHERE Livsmedelsnummer = "2173";
UPDATE livs SET Huvudgrupp = "Korvrätter" WHERE Livsmedelsnumm

UPDATE livs SET Huvudgrupp = "Mjukt bröd" WHERE Livsmedelsnummer = "3796";
UPDATE livs SET Huvudgrupp = "Mjukt bröd" WHERE Livsmedelsnummer = "3797";
UPDATE livs SET Huvudgrupp = "Mjukt bröd" WHERE Livsmedelsnummer = "3798";
UPDATE livs SET Huvudgrupp = "Baljväxter (bönor, linser och ärter)" WHERE Livsmedelsnummer = "3815";
UPDATE livs SET Huvudgrupp = "Baljväxter (bönor, linser och ärter)" WHERE Livsmedelsnummer = "3816";
UPDATE livs SET Huvudgrupp = "Baljväxter (bönor, linser och ärter)" WHERE Livsmedelsnummer = "3817";
UPDATE livs SET Huvudgrupp = "Mjukt bröd" WHERE Livsmedelsnummer = "3818";
UPDATE livs SET Huvudgrupp = "Mjukt bröd" WHERE Livsmedelsnummer = "3819";
UPDATE livs SET Huvudgrupp = "Grönsaks- rotfrukts- baljväxträtter och produkter" WHERE Livsmedelsnummer = "3820";
UPDATE livs SET Huvudgrupp = "Baljväxter (bönor, linser och ärter)" WHERE Livsmedelsnummer = "3821";
UPDATE livs SET Huvudgrupp = "Baljväxter (bönor, linser och ärter)" WHERE Livsmedelsnummer = "3822";
UPDATE

UPDATE livs SET Huvudgrupp = "Sås dressing majonnäs" WHERE Livsmedelsnummer = "5916";
UPDATE livs SET Huvudgrupp = "Fisk färsk fryst kokt" WHERE Livsmedelsnummer = "5926";
UPDATE livs SET Huvudgrupp = "Matgryn" WHERE Livsmedelsnummer = "5935";
UPDATE livs SET Huvudgrupp = "Fisk färsk fryst kokt" WHERE Livsmedelsnummer = "5936";
UPDATE livs SET Huvudgrupp = "Lightdrycker u energi" WHERE Livsmedelsnummer = "5942";
UPDATE livs SET Huvudgrupp = "Vegetabiliska produkter och mjölkersättning" WHERE Livsmedelsnummer = "5956";
UPDATE livs SET Huvudgrupp = "Vegetabiliska produkter och mjölkersättning" WHERE Livsmedelsnummer = "5957";
UPDATE livs SET Huvudgrupp = "Vegetabiliska produkter och mjölkersättning" WHERE Livsmedelsnummer = "5961";
UPDATE livs SET Huvudgrupp = "Vegetabiliska produkter och mjölkersättning" WHERE Livsmedelsnummer = "5962";
UPDATE livs SET Huvudgrupp = "Vegetabiliska produkter och mjölkersättning" WHERE Livsmedelsnummer = "5963";
UPDATE livs SET Huvudgrupp = "Vegetabiliska 

In [12]:
c = b[:,1:]
unique, counts = np.unique(c, return_counts=True) #https://stackoverflow.com/questions/28663856/how-to-count-the-occurrence-of-certain-item-in-an-ndarray-in-python
huvudgrupper = dict(zip(unique, counts))
print(huvudgrupper)
len(unique)

{'Godis ej choklad': 11, 'Kakaoprodukter': 2, 'Blodmat': 2, 'Salt': 4, 'Nötter frön': 22, 'Sylt marmelad gelé äppelmos o dyl': 18, 'Soja-  veteprotein o quorn': 3, 'Smör': 3, 'Fisk färsk fryst kokt': 68, 'Fisk o skaldjursprodukter o rätter': 69, 'Köttprodukter kötträtter': 80, 'Socker sirap honung': 6, 'Sås dressing majonnäs': 79, 'Ris risnudlar': 25, 'Smörgåskex': 7, 'Sockerfritt godis': 1, 'Öl': 4, 'Välling': 12, 'Skaldjur bläckfisk färsk fryst kokt': 14, 'Mjöl stärkelse kli': 29, 'Smaksatt fil yoghurt': 12, 'Övrigt': 3, 'Choklad': 7, 'Fågel': 39, 'Kaffe': 5, 'Kryddor': 1, 'Söta soppor kräm o efterrättssås': 26, 'Pastarätter': 12, 'Äggprodukter o rätter': 6, 'Ägg': 9, 'Hamburgare med  bröd (kött, fisk, fågel, vegetarisk)': 5, 'Soppa mat': 47, 'Senap ketchup HP-sås soja *smaksättare*': 11, 'Pannkakor, våfflor, crêpes': 13, 'Frukt färsk fryst': 42, 'Saft läsk cider u alkohol': 19, 'Svamp': 9, 'Blodprodukter o rätter': 8, 'Fågelprodukter o rätter': 5, 'Frukt o bär konserverade': 17, 'Fr

118

In [13]:
import csv

Det är lättare att stoppa här, och lämna analysen av kategorierna till ett annat underprojekt.

Jag har blivit av med de flesta citationstecken och sånt, men när det är parenteser i gruppnamnet verkar den lägga till citationstecken runt strängen...

In [15]:
with open('huvudgrupper.csv', 'w') as csv_file: #https://stackoverflow.com/questions/8685809/python-writing-a-dictionary-to-a-csv-file-with-one-line-for-every-key-value
    #Filen behövde öppnas med w och inte wb för det är ingen binärfil.
    writer = csv.writer(csv_file)
    for key, value in huvudgrupper.items():
        writer.writerow([key, value])
        #print([key, value])